In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import collections

In [ ]:
keys = ['Carpark Name', 'Carpark Address', 'Mon-Fri before 5/6pm', 'Mon-Fri after 5/6pm', 'Sat', 'Sun/Public Holiday']
dict = collections.OrderedDict((key,[]) for key in keys)

location = 'na'
address = 'na'
full_table = 0
url = 'na'

In [ ]:
def readurl(url):
    """
    This function takes the website url and returns the full table containing the required info
    """
    dfs = pd.read_html(url)
    data = dfs[6]
    global full_table
    full_table = data.iloc[:,0]
    return full_table

In [ ]:
def add_and_loc(url):
    """
    This function takes the website url and returns the location and address of the carpark
    """
    resp = requests.get(url)
    html_doc = resp.text
    soup = BeautifulSoup(html_doc, 'html.parser')

    title = soup.find(class_="grayboxborder")
    global location
    location = title.text.strip()
    loc = location.split("\n")
    location = loc[0]
    global address
    address = loc[1].rstrip()
    return location, address

In [ ]:
def append(table,location,address):
    """
    This function takes all the info, location and address of the carpark and appends it to the dictionary
    """
    global dict
    dict['Carpark Name'].append(location)
    dict['Carpark Address'].append(address)
    dict['Mon-Fri before 5/6pm'].append(full_table[1].replace('$','\$'))
    dict['Mon-Fri after 5/6pm'].append(full_table[3].replace('$','\$'))
    dict['Sat'].append(full_table[5].replace('$','\$'))
    dict['Sun/Public Holiday'].append(full_table[7].replace('$','\$'))

In [ ]:
#repeat the scraping across webpages, up to 914
for i in range (1,915):
    try:
        front_url = 'http://www.sgcarmart.com/news/carpark_index.php?ID='
        back_url = '&LOC=all&TYP=carpark&SRH=#carparkrates'
        url = front_url + str("%03d" % i) + back_url
        readurl(url)
        add_and_loc(url)
        append(full_table,location,address)
    except AttributeError:
        print(url + ' val')
        i += 1
        continue
    except ValueError:
        print(url + ' val')
        i += 1
        continue
    except IndexError:
        print(url + ' index')
        i += 1
        continue

In [ ]:
df = pd.DataFrame.from_dict(dict, orient='index')
dft = df.transpose()
df.to_csv('Parking_20170918.csv', index=False)